<a href="https://colab.research.google.com/github/viett887g/baitapkinang2/blob/main/baitapkinaglogictic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from scipy import optimize
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [148]:
def readData(filePath: str, filename: str):
    data = np.loadtxt(os.path.join(filePath, filename), delimiter = ',')
    data=preprocessing.MinMaxScaler().fit_transform(data)
    #data=preprocessing.StandardScaler().fit_transform(data)
    X = data[:,:-1]
    y = data[:, -1]
    m = X.shape[0]
    n = X.shape[1]
    X = np.reshape(X, (m,n))
    y = np.reshape(y, (m,1))
    #Them cot x0 = 1 vao X
    x0 = np.ones((m,1))
    X = np.column_stack([x0, X])
    return X, y

def featureVectorScaling(data):
    avg = np.mean(data)
    sln = data.max()
    snn = data.min()
    data_scl = (data - avg)/(sln - snn)
    print(data_scl[1])
    return data_scl

def normalizeData(X):
    X_scl = X[:, 0]
    for i in range(1, X.shape[1]):
        scl = featureVectorScaling(X[:, i])
        X_scl = np.column_stack([X_scl, scl])
    return X_scl

#Day chinh la ham  hw(X)
def sigmoid(X, w):
    result = 1/(1 + np.exp(-np.dot(X, w)))
    return result

def loss(w, X, y):
    m = X.shape[0]
    h = sigmoid(X, w)
    result = (-1 / m) * np.sum(np.dot(y.T, np.log(h)) + np.dot((1 - y).T, np.log(1 - h)))
    return result

def scipy(X,y,w,n_iters):
    result = optimize.minimize(fun=loss, x0=w, args=(X,y),method='L-BFGS-B',options={"maxiter":n_iters} )
    w_optimal = result.x
    J_optimal = result.fun
    return w_optimal, J_optimal

def scipy2(X,y,w,n_iters):
    result = optimize.minimize(fun=loss, x0=w, args=(X,y),method='Powell',options={"maxiter":n_iters} )
    w_optimal2 = result.x
    J_optimal2 = result.fun
    return w_optimal2, J_optimal2  

def gradient(X, y, w):
    m = X.shape[0]
    result = (1/m)*np.dot(X.T, sigmoid(X, w) - y)
    return result

def gradientDescent(X, y, w, alpha, n_iters):
    w_optimal = w.copy()
    J_history = []
    for i in range(n_iters):
        w_optimal = w_optimal - alpha*gradient(X, y, w_optimal)
        J_history.append(loss(X, y, w_optimal))
    return w_optimal, J_history

#Hàm dự đoán nếu y_pred >=0.5 làm tròn thành 1, ngược lại là 0
def predict(y_pred):
    return np.rint(y_pred)

def acc_score(y, y_hat):
    result = accuracy_score(y, y_hat)
    return  result


def nggeg(X,y):
  model = LogisticRegression().fit(X,y)
  w_otp0 = model.coef_
  return w_otp0

def kfolk(X):
  kf = KFold(n_splits=10)
  kf.get_n_splits(X)
  KFold(n_splits=10, random_state=None, shuffle=False)

In [123]:
>>> from sklearn import datasets, linear_model
>>> from sklearn.model_selection import cross_validate
>>> from sklearn.metrics import make_scorer
>>> from sklearn.metrics import confusion_matrix
>>> from sklearn.svm import LinearSVC

In [140]:
def crossvalidate(X,y):
  diabetes = datasets.load_diabetes()
  lasso = linear_model.Lasso()
  cv_results = cross_validate(lasso, X, y, cv=4)
  print(cv_results['test_score'])

In [134]:
>>> from sklearn import datasets, linear_model
>>> from sklearn.model_selection import cross_val_score

In [156]:
def cross_valscore(X,y):
  diabetes = datasets.load_diabetes()
  lasso = linear_model.Lasso()
  print(cross_val_score(lasso, X, y))

In [150]:
>>> from sklearn import datasets, linear_model
>>> from sklearn.model_selection import cross_val_predict

In [199]:
def cross_valpredict1(X,y):
  diabetes = datasets.load_diabetes()

  cv = KFold(n_splits=4)
  lasso = linear_model.Lasso()
  y_pred = cross_val_predict(lasso, X, y, cv=cv)
  return y_pred

In [201]:
def main():
    X, y = readData('/content','ex2data1.txt')
    X = normalizeData(X)
    n = X.shape[1]
    w = np.zeros((n, 1))
    alpha = 0.01
    n_iters = 2000
    #Chia train - test
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.30,random_state=15)
    w_optimal, J_optimal=scipy(X_train,y_train,w,n_iters)
    #w_opt, J_hist = gradientDescent(X_train, y_train, w, alpha, n_iters)
    w_optimal2, J_optimal2=scipy2(X_train,y_train,w,n_iters)
    print('Trong so w toi uu la L-BFGS-B: ', w_optimal)
    print('tGia tri Loss toi uu: ', J_optimal)
    print('tTrong so w toi uu la Powell: ', w_optimal2)
    print('tGia tri Loss toi uu: ', J_optimal2)
    y_hat = predict(sigmoid(X_test,w_optimal2))
    
    print('Sử dụng sklearn, Acc: ',acc_score(y_test, y_hat))
    print("....\nmô hình LogisticRegression")
    w_otp0=nggeg(X_train,y_train)
    print("trong so toi uu ",w_otp0)
    y_hat4=predict(sigmoid(X_test,w_otp0[0]))
    print('Sử dụng sklearn, Acc: ',acc_score(y_test, y_hat4))

    kfolk(X_test)
    print("Thực hiện huấn luyện mô hình với cross_validate;")
    crossvalidate(X_test,y_test)
    print("\nhiện huấn luyện mô hình với cross_val_score")
    cross_valscore(X_test,y_test)

    y_pred=cross_valpredict1(X_test,y_test)
    print("\nSử dụng cross_val_predict để dự đoán.",y_pred)

if __name__ == "__main__":
    main()

-0.5067801656080071
-0.3270580397857181
Trong so w toi uu la L-BFGS-B:  [ 2.83260864 16.76544116 21.14450233]
tGia tri Loss toi uu:  0.18479478833312113
tTrong so w toi uu la Powell:  [ 2.83085511 16.75294021 21.12525394]
tGia tri Loss toi uu:  0.18479487372203188
Sử dụng sklearn, Acc:  0.9
....
mô hình LogisticRegression
trong so toi uu  [[-1.18859659e-05  2.38821619e+00  2.56233703e+00]]
Sử dụng sklearn, Acc:  0.8666666666666667
Thực hiện huấn luyện mô hình với cross_validate;
[-0.20661157 -0.0137741  -0.01890359 -0.50409578]

hiện huấn luyện mô hình với cross_val_score
[-0.17361111 -0.78125    -0.3125      0.          0.        ]

Sử dụng cross_val_predict để dự đoán. [0.72727273 0.72727273 0.72727273 0.72727273 0.72727273 0.72727273
 0.72727273 0.72727273 0.68181818 0.68181818 0.68181818 0.68181818
 0.68181818 0.68181818 0.68181818 0.68181818 0.65217391 0.65217391
 0.65217391 0.65217391 0.65217391 0.65217391 0.65217391 0.60869565
 0.60869565 0.60869565 0.60869565 0.60869565 0.60869

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
